In [2]:
using WGLMakie
using GLMakie
using Graphs
using JSServe
using NetworkLayout
using GraphMakie
using CSV
using DataFrames
using LinearAlgebra

# Let's first get the graph structure.

In [3]:
flows = DataFrame(CSV.File("whole_cell_flow_data.csv"))

Row,Column1,data
,String,Float64
1,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),3851.0
2,1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),16095.0
3,1PFRUCTPHOSN-RXN,1.23812e6
4,1TRANSKETO-RXN (reverse),7974.0
5,2-DEHYDROPANTOATE-REDUCT-RXN (reverse),1131.0
6,2-ISOPROPYLMALATESYN-RXN,56297.0
7,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,300.0
8,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,300.0
9,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,300.0


In [4]:
stoichiometry = DataFrame(CSV.File("whole_cell_s_matrix (1).csv"))
metabolite_names = stoichiometry[:, "Column1"]
reaction_names = names(stoichiometry)[2:end]

n_mets = length(metabolite_names)
n_rxns = length(reaction_names)

S = Matrix(stoichiometry[:, 2:end])

489×470 Matrix{Int64}:
  1   1  0  0   0   0   0  0  0   0  0  …  0   0  0   0   0  0  0   0  0   0
 -1  -1  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
 -1  -1  1  0  -1   1  -1  1  1  -1  1     0  -1  0   1   0  0  0   1  0   1
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   1   0   1  0  0   1  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0  -1   0  -1  0  0  -1  0  …  0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   1  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0  …  0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0  -1   1  0  0   1  0     0   0  0  -1  -1  0  0  -1  0  -1
  0   0  0  0   0   0  -1  0  0  -1  0     0   0  0  

In [5]:
B = S .!= 0
B

489×470 BitMatrix:
 1  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  0  1  1  1  1  1  1  1  0  1     1  0  0  1  0  1  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  1  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  1  0  0  1  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1     1  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  0  0  1  0  0  0     0  0  0  0  0  1  1  0  0  1  0  1
 0  0  0  0  0  0  1  0  0  1  0  0  0     0 

In [6]:
B * Transpose(B)

489×489 Matrix{Int64}:
 38  36   33  0   2   1  0   5   1  0  …  1  1  0  0  0  0  0  0  0  0  0  0
 36  36   31  0   1   1  0   5   1  0     1  1  0  0  0  0  0  0  0  0  0  0
 33  31  236  0  30  29  1  42  11  2     1  2  2  1  2  1  3  2  1  0  1  0
  0   0    0  1   0   0  0   0   0  0     0  0  0  0  0  0  0  0  0  0  0  0
  2   1   30  0  33  32  0   2   0  1     0  0  1  1  1  1  1  1  0  0  0  0
  1   1   29  0  32  32  0   2   0  1  …  0  0  1  1  1  1  1  1  0  0  0  0
  0   0    1  0   0   0  1   0   0  0     0  0  0  0  0  0  0  0  0  0  0  0
  5   5   42  0   2   2  0  50   8  0     0  0  1  0  1  0  2  0  0  0  0  0
  1   1   11  0   0   0  0   8  18  0     0  0  0  0  0  0  0  0  0  0  0  0
  0   0    2  0   1   1  0   0   0  2     0  0  0  0  0  0  0  0  0  0  0  0
  0   0    2  0   1   1  0   0   0  2  …  0  0  0  0  0  0  0  0  0  0  0  0
  6   6   51  1   4   4  1   7   3  0     1  1  0  1  0  1  0  1  0  1  1  0
  0   0    4  0   2   2  0   1   0  0     0  0  0  0 

In [7]:
node_freq = diag(B * Transpose(B))
node_freq

489-element Vector{Int64}:
  38
  36
 236
   1
  33
  32
   1
  50
  18
   2
   2
 120
   8
   ⋮
   2
   2
   2
   2
   2
   2
   3
   4
   2
   2
   2
   1

In [8]:
pair_freq_tri = LowerTriangular(B * Transpose(B) - Diagonal(node_freq))
pair_indices = findall(pair_freq_tri .!= 0)
pair_counts = pair_freq_tri[pair_indices]

2674-element Vector{Int64}:
 36
 33
  2
  1
  5
  1
  6
  1
  1
  1
  1
  2
  1
  ⋮
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1

In [9]:
pair_max_threshold = 2

removed_pairs = pair_indices[pair_counts .> pair_max_threshold]
n_pairs = length(removed_pairs)

node_max_threshold = 20
removed_nodes = node_freq .> node_max_threshold

489-element BitVector:
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 1
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [10]:
metabolite_names[removed_nodes]

13-element Vector{String}:
 "NAD[c]"
 "NADH[c]"
 "PROTON[c]"
 "NADP[c]"
 "NADPH[c]"
 "CARBON-DIOXIDE[c]"
 "WATER[c]"
 "ATP[c]"
 "ADP[c]"
 "CO-A[c]"
 "Pi[c]"
 "GLT[c]"
 "PPI[c]"

In [11]:
R = Int.(zeros(n_pairs, n_mets))

for i in 1:n_pairs
    pair = removed_pairs[i]
    R[i, pair[1]] = 1
    R[i, pair[2]] = 1
end

Sm = copy(S)

for j in 1:n_rxns
    
    matched_pair_pair_indices = R * B[:, j] .== 2

    matched_met_indices = unique(getindex.(removed_pairs[matched_pair_pair_indices], [1 2]))
    
    Sm[matched_met_indices, j] .= 0
    
end

Sm[removed_nodes, :] .= 0

13×470 view(::Matrix{Int64}, [1, 2, 3, 5, 6, 8, 12, 21, 23, 34, 46, 57, 63], :) with eltype Int64:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     

In [12]:
G = SimpleDiGraph(length(metabolite_names) + length(reaction_names))


LoadError: UndefVarError: `SimpleDiGraph` not defined

In [13]:
reactions = flows[:, "Column1"]
flows = normalize(log.(flows[:, "data"])) * 100
weights = Dict()


for i in 1:n_mets
    
    for j in 1:n_rxns
        
        # check if 
        if Sm[i, j] > 0
            # add_edge!(G, i, n_mets + j)
            add_edge!(G, n_mets + j, i)
        elseif Sm[i,j] < 0
            add_edge!(G, i, n_mets + j)
        end
        
        # Update edge weight dictionary
        weights[n_mets + j] = flows[j]
    end
end


LoadError: UndefVarError: `G` not defined

In [14]:
big_network_nodes = reduce(vcat, connected_components(G)[length.(connected_components(G)) .> 100])
Gb = induced_subgraph(G, big_network_nodes)[1]

LoadError: UndefVarError: `connected_components` not defined

## Reduce paths between nodes with degree >2

In [15]:
path_nodes = findall(degree(Gb) .== 2)
fork_nodes = findall(degree(Gb) .!= 2)

found = false

node_group = Int.(zeros(length(vertices(Gb))))
cur_group = 1


for node in path_nodes
    # println(all_neighbors(Gb, node))
    
    # Start path if either connection is not path
    side_neighbors = all_neighbors(Gb, node)
    found = false
    # println("starting node $node")
    # println(cur_group)
    
    if side_neighbors[1] in fork_nodes && side_neighbors[2] in fork_nodes
        # println("double")
        # add to path
        continue
        
    elseif side_neighbors[1] in fork_nodes && node_group[node] == 0
        
        # go through until fork is encountered and add to path
        cur_node = node
        prev_node = side_neighbors[1]
        start_node = side_neighbors[1]
        
        # println("left, node $start_node is start fork, going to node $cur_node")
        
        while found == false
            
            node_group[cur_node] = cur_group
            
            side_neighbors = all_neighbors(Gb, cur_node)
            
            # println("neighbors for $cur_node are $side_neighbors")
            
            other_neighbor = setdiff(side_neighbors, prev_node)[1]
            
            if other_neighbor in fork_nodes
                # println("node $other_neighbor is end fork")
                end_node = other_neighbor
                
                found = true
            else
                # println("traversing from node $cur_node to node $other_neighbor")
                prev_node = cur_node
                cur_node = other_neighbor
            end
        end
        
        cur_group += 1
        
    elseif side_neighbors[2] in fork_nodes && node_group[node] == 0
        # println("right")
        # go through until fork is encountered and add to path
        
        # go through until fork is encountered and add to path
        cur_node = node
        prev_node = side_neighbors[2]
        start_node = side_neighbors[2]
        
        # println("left, node $start_node is start fork, going to node $cur_node")
        
        while found == false
            
            node_group[cur_node] = cur_group
            
            side_neighbors = all_neighbors(Gb, cur_node)
            
            # println("neighbors for $cur_node are $side_neighbors")
            
            other_neighbor = setdiff(side_neighbors, prev_node)[1]
            
            if other_neighbor in fork_nodes
                # println("node $other_neighbor is end fork")
                end_node = other_neighbor
                
                found = true
            else
                # println("traversing from node $cur_node to node $other_neighbor")
                prev_node = cur_node
                cur_node = other_neighbor
            end
        end
        
        
        cur_group += 1
        
    else
        # println("none")
        continue
    end
        
end

LoadError: UndefVarError: `degree` not defined

## Generate annotations

In [16]:
all_names = [metabolite_names; reaction_names]

graph_node_labels = all_names[big_network_nodes[1:length(vertices(Gb))]]

graph_node_colors = repeat([colorant"blue"], length(graph_node_labels))
graph_node_colors[big_network_nodes .> length(metabolite_names)] .= colorant"blue"
graph_node_colors[big_network_nodes .<= length(metabolite_names)] .= colorant"red"
graph_node_colors[node_group .!= 0] .= colorant"yellow"


LoadError: UndefVarError: `big_network_nodes` not defined

In [17]:
weight_vec = []
for (i, edge) in enumerate(edges(Gb))
    source = big_network_nodes[src(edge)]
    dest = big_network_nodes[dst(edge)]
    
    if haskey(weights, source)
        push!(weight_vec, weights[source])
    elseif haskey(weights, dest)
        push!(weight_vec, weights[dest])
    else
        print(edge)
        error("Invalid edge")
    end
    
end

LoadError: UndefVarError: `edges` not defined

In [18]:
Page(exportable=true, offline=true)

# WGLMakie.activate!()
GLMakie.activate!(inline=false)

screen = GLMakie.Screen(float=true)
set_theme!(resolution=(500, 300))
f, ax, p = graphplot(Gb, layout=Stress(dim=2), 
    node_color = graph_node_colors,
    edge_width=weight_vec)

scene = ax.scene

on(events(scene).mousebutton) do event
    if event.button == Mouse.left
        if event.action == Mouse.press || event.action == Mouse.release
            println("Graph axis position: $(event)")
        end
    end
end


display(screen,f)


LoadError: UndefVarError: `graph_node_colors` not defined

In [ ]:
scene.plots[2][1]

In [ ]:
Page(exportable=true, offline=true)

WGLMakie.activate!()


# set_theme!(resolution=(1000, 800))

g = wheel_digraph(10)
f, ax, p = graphplot(g, node_size=30, node_color=[colorant"red" for i in 1:nv(g)])

# display(screen,f)

open("index.html", "w") do io
    println(io, """
    <html>
        <head>
        </head>
        <body>
    """)
    Page(exportable=true, offline=true)
    # Then, you can just inline plots or whatever you want :)
    # or anything else from JSServe, or that can be displayed as html:
    show(io, MIME"text/html"(), graphplot(Gb, layout=Stress(dim=3), node_color = graph_node_colors,))
    println(io, """
        </body>
    </html>
    """)
end

In [ ]:
# Generate random edge weights
weights = rand(length(edges(Gb)))

# Create a layout algorithm using the weights
layout = layoutalgorithm(G, :Stress, edge_weights = weights)
